# Here is my attempt at crypto-trading. I utilize datetime, neural nets, supervised, and unsupervised learning to code an algorithm to automatically trade cryptocurrency.

Hypothesis: With the assistance of supervised, unsupervised learning, along with neural networks, I wil be able to predict the movement of the close price of bitcoin 60 mins into the future.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from pandas_datareader import data
import datetime as dt
from datetime import datetime
import seaborn as sns
from scipy.stats import boxcox
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential,model_from_json
from tensorflow.keras.layers import Dense, LSTM, Dropout, ConvLSTM2D, Flatten, RepeatVector,TimeDistributed
from tensorflow.keras.utils import to_categorical
import umap
import pickle

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [ ]:
#Function I will use so I'm saving them for later
#This function splits data into X and y for neural networks and makes X into a 3d array to be acceptable for LSTM models
def convert_data(data, step):
  X, y = [], []
  for i in range(len(data)-step):
    d = i + step
    X.append(data[i:d,:-1])
    y.append(data[d,-1])
  return np.array(X), np.array(y)


# Training data is found from kaggle. Link is below. I'm using the btc_usd one
#link: https://www.kaggle.com/tencars/392-crypto-currency-pairs-at-minute-resolution



In [ ]:
#I saved the data here
df = pd.read_csv(r"/content/drive/My Drive/Crypto/btcusd.csv")
df = df.groupby('time').mean()
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2841761 entries, 1364774820000 to 1593425100000
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   open    float64
 1   close   float64
 2   high    float64
 3   low     float64
 4   volume  float64
dtypes: float64(5)
memory usage: 130.1 MB


None

In [ ]:
#Check for nulls to see. YAY! No nulls!
df.isna().mean()

open      0.0
close     0.0
high      0.0
low       0.0
volume    0.0
dtype: float64

In [ ]:
df.head()

,open,close,high,low,volume
time,,,,,
1364774820000,93.25,93.30,93.30,93.25,93.300000
1364774880000,100.00,100.00,100.00,100.00,93.300000
1364774940000,93.30,93.30,93.30,93.30,33.676862
1364775060000,93.35,93.47,93.47,93.35,20.000000
1364775120000,93.47,93.47,93.47,93.47,2.021627


Please comment out this code if you want to see the effect. Like I said above, I ran out of RAM and can't download more

In [ ]:
# plot_df = df.copy().reset_index()
# plot_df['time'] = pd.to_datetime(plot_df['time'],unit = "ms")
# plt.plot("time","close", data = plot_df, linewidth = .25)
# plt.show()

In [ ]:
# I will add various RMAs in an attempt to give the computer a better chance at understanding and predicting when to buy/sell
# The point of these is to give models extra help in predicting the price going up or down 60 mins down the road
df['High 8 RMA'] = df['close'].rolling(window=8).mean()
df['High 13 RMA'] = df['close'].rolling(window=13).mean()
df['High 21 RMA'] = df['close'].rolling(window=21).mean()
df['High 55 RMA'] = df['close'].rolling(window=55).mean()
df['Bollinger High'] = df['High 13 RMA'] +df['close'].rolling(2).std()
df['Bollinger Low'] =  df['High 13 RMA'] - df['close'].rolling(2).std()
df['returns'] = df['close'].pct_change(60)

#What I'm trying to predict
#1st strategy loses me money: price going up or down in 60 mins
df['target']=(df['returns'].shift(-60)>0).astype(int)

'''
THESE ARE SOLELY FOR ME. THEY ARE NOT PART OF THE PROJECT BUT THINGS FOR ME TO PLAY WITH ON MY OWN. I DO NOT USE THESE LATER ON
BUT PLAN ON USING THESE PERSONALLY IN THE FUTURE SO THEY'RE HERE. IF YOU HAVE ANY SUGGESTIONS OF OTHER STRATEGIES, PLEASE SHARE :D



#2nd strategy is if the Bollinger low band crosses the price
#df['target'] = ((df['close']>df['Bolllinger Low']).shift(-60)>0).astype(int)

#3rd Strategy aka the youtube guru strategy. This is something I found on youtube and thought it would be fun to backtest and try
#df['target']=((df['High 8 RMA']>df['High 13 RMA']>df['High 21 RMA']>df['High 55 RMA']).shift(-60)>0).astype(int)
'''

"\nTHESE ARE SOLELY FOR ME. THEY ARE NOT PART OF THE PROJECT BUT THINGS FOR ME TO PLAY WITH ON MY OWN. I DO NOT USE THESE LATER ON\nBUT PLAN ON USING THESE PERSONALLY IN THE FUTURE SO THEY'RE HERE. IF YOU HAVE ANY SUGGESTIONS OF OTHER STRATEGIES, PLEASE SHARE :D\n\n\n\n#2nd strategy is if the Bollinger low band crosses the price\n#df['target'] = ((df['close']>df['Bolllinger Low']).shift(-60)>0).astype(int)\n\n#3rd Strategy aka the youtube guru strategy. This is something I found on youtube and thought it would be fun to backtest and try\n#df['target']=((df['High 8 RMA']>df['High 13 RMA']>df['High 21 RMA']>df['High 55 RMA']).shift(-60)>0).astype(int)\n"

Why am I going from 2017 when I have more data?

Link:



1) Before 2017 and after 2017, there is a massive behavior change in the movement of the price so I want to capture current behavior

2) It's easier on my ram to actually run different types of machine learning methods on less data. Including all of the data has crashed my kernal multiple times

In [ ]:
#now I'm going from 2017 to June 2020
df = df.iloc[-1051200:].copy()
#Making a time variable for visualizations
df['time'] = df.index
x = pd.to_datetime(df['time'],unit = "ms")
df.drop(columns = 'time', inplace = True)
#Moving stuff to my next notebook
#x.to_csv("/content/drive/My Drive/Crypto/time.csv")

In [ ]:
#Much better. Now it's time to re-check for NAs and make sure that didn't mess anything up

Please comment out this code if you want to see the effect. Like I said above, I ran out of RAM and can't download more

In [ ]:
# #Cool! Now let's check out how the line goes for open close high and low
# df.isna().sum()

Please comment out this code if you want to see the effect. Like I said above, I ran out of RAM and can't download more

I've also added links

In [ ]:
# #This makes it look like the difference isn't really there. I'm curious how much of a difference there is between all of these

# # Link: "https://github.com/AsterLaoWhy/Thinkful/blob/master/Prices.png"

# plot_df = df.copy().reset_index()
# plot_df['time'] = pd.to_datetime(plot_df['time'],unit = "ms")
# plt.figure(figsize = (12,6))
# plt.plot("time","open", data = plot_df, linewidth = .25)
# plt.plot("time","close", data = plot_df, linewidth = .25)
# plt.plot("time","high", data = plot_df, linewidth = .25)
# plt.plot("time","low", data = plot_df, linewidth = .25)
# plt.legend()
# plt.show()

Please comment out this code if you want to see the effect. Like I said above, I ran out of RAM and can't download more

I've also added links

In [ ]:
# #That's weird. None of the volume is 0
# # Link: "https://github.com/AsterLaoWhy/Thinkful/blob/master/Volume.png"

# plt.plot("time","volume",data = plot_df)
# plt.title("Volume Traded")
# plt.show()


Please comment out this code if you want to see the effect. Like I said above, I ran out of RAM and can't download more.

I've also added links

In [ ]:
# #Returns and close
# #1 is buy and 0 is sell/do nothing

# # link:"https://github.com/AsterLaoWhy/Thinkful/commit/cadc8011516367d57e79389d619941b0c76d4bc1"
# plt.figure(figsize=(12,6))
# plt.scatter("time","target", data = plot_df[-60:], linewidth = 1)
# plt.legend()
# plt.show()


# # link: "https://github.com/AsterLaoWhy/Thinkful/blob/master/1%20day%20target.png"
# plt.figure(figsize=(12,6))
# plt.scatter("time","target", data = plot_df[-1440:], linewidth = 1)
# plt.legend()
# plt.show()

Link to day:

Link to hour:

# At this point, things I can tune are
 - the time window of data being taken into the model
 - Which moving average the Bollinger bands are taking into account
 - Predicing the 4 moving average split rather than future price crosses

### Let's run some classification supervised learning models to see how it works. I will try LogRegression, gradient boosted classification, neural nets 

In [ ]:
# scaler = MinMaxScaler()
# x = df.drop(['returns', 'target'], axis = 1)
# y = df['target']
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 69, shuffle = False)
# X_train= scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
# y_test.to_csv('/content/drive/My Drive/Crypto/y_test.csv', index=False)  
# np.savetxt("/content/drive/My Drive/Crypto/X_test.txt",X_test)


In [ ]:
df.columns

Index(['open', 'close', 'high', 'low', 'volume', 'High 8 RMA', 'High 13 RMA',
       'High 21 RMA', 'High 55 RMA', 'Bollinger High', 'Bollinger Low',
       'returns', 'target'],
      dtype='object')

In [ ]:
#Due to Ram constraints, please see my second notebook for model comparison

# LGR = LogisticRegression(class_weight = 'balanced', random_state = 69)
# LGR.fit(X_train, y_train)
# predictions = LGR.predict(X_test)
# np.savetxt("/content/drive/My Drive/Crypto/LGR_predictions.txt", predictions)
# pickle.dump(LGR, open("/content/drive/My Drive/Crypto/CryptoLGR_model.sav", "wb"))

In [ ]:
# Due to Ram constraints, please see my second notebook for model comparison
# XGBC = XGBClassifier(max_depth = 2, learning_rate=.5, random_state=69)
# XGBC.fit(X_train, y_train)
# predictions = XGBC.predict(X_test)
# np.savetxt("/content/drive/My Drive/Crypto/XGB_predictions.txt", predictions)
# pickle.dump(XGBC, open("/content/drive/My Drive/Crypto/XGBC_model.sav", "wb"))

In [ ]:
lstm_df = df[df.columns.drop(list(df.filter(regex='t-')))]
# lstm_df.to_csv('/content/drive/My Drive/Crypto/lstm_df.csv', index=False)
X_train, y_train = convert_data(lstm_df[:788400].values, 60)
X_test, y_test = convert_data(lstm_df[788401:].values, 60)

Here is the LSTM model. Uncomment the code to run it. Running this kills my kernal if I run any other model/unsupervised learning

In [ ]:
# #LSTM network
# #for LSTM start , input_shape=(X_train.shape[1], X_train.shape[2]
# # Answer below:
# model = Sequential()
# #Throw in some LSTM
# model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dropout(0.2))
# model.add(LSTM(75))
# model.add(Dropout(0.2))
# #Finish out the model
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='softmax'))


In [ ]:
# #Adam Spannbauer coming back as an optimizer
# # I legit run out of ram after epoch 3 so I'm only doing 2 for that reason. Something to improve would be to have a better computer so I don't max out on ram
# #But the metrics are all "102s 39ms/step - loss: 7.5652 - accuracy: 0.5039 - val_loss: 7.4533 - val_accuracy: 0.5112"
# model.compile(optimizer = "adam", loss='binary_crossentropy',
#               metrics=['accuracy'])
# model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=300, epochs = 2)

# model.save("/content/drive/My Drive/Crypto/lstm_model.h5")
# model.save_weights("/content/drive/My Drive/Crypto/lstm_model_weights.h5")
# predictions = model.predict(X_test)
# np.savetxt("/content/drive/My Drive/Crypto/LSTM_X.txt", predictions)

In [ ]:
#Now let's jump over to the other notebook for supervised learning/LSTM model comparisons...

# Man, neural nets are performing bad. It's not that my features are bad, I don't ever blame myself... "All this hype about neural nets for nothing"- Jakob Salomonsson

In [ ]:
scaler = MinMaxScaler()
x = df.drop(['returns', 'target'], axis = 1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 69, shuffle = False)
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Maybe some unsupervised learning will help these models go super sayian
#Let's try PCA and UMAP
#For the PCA, I'm taking in already split data . I could take in the whole df minus target to fit PCA and then split. 

In [ ]:
X_train.shape

Here I use the PCAed features to try to predict the stock movement

In [ ]:
# # Trying 2,3,4 features. Want to see if this feature reduction will help the models will go further beyond
# #2 components LGR
# LGR = LogisticRegression(class_weight = 'balanced', random_state = 69)
# pca = PCA(n_components=2)
# PCA2_train = pca.fit_transform(X_train)
# PCA2_test = pca.fit_transform(X_test)
# LGR.fit(PCA2_train, y_train)
# predictions = LGR.predict(PCA2_test)
# pickle.dump(LGR, open("/content/drive/My Drive/Crypto/CryptoLGR2_model.sav", "wb"))
# np.savetxt("/content/drive/My Drive/Crypto/LGR2_X.txt",predictions)

# #2 Component XGBC
# XGBC = XGBClassifier(max_depth = 2, learning_rate=.5, random_state=69)
# XGBC.fit(PCA2_train, y_train)
# predictions = XGBC.predict(PCA2_test)
# np.savetxt("/content/drive/My Drive/Crypto/XGB2_predictions.txt", predictions)
# pickle.dump(XGBC, open("/content/drive/My Drive/Crypto/XGBC2_model.sav", "wb"))


# #---------------------------------------------------


# #3 components
# LGR = LogisticRegression(class_weight = 'balanced', random_state = 69)
# pca = PCA(n_components=3)
# PCA3_train = pca.fit_transform(X_train)
# PCA3_test = pca.fit_transform(X_test)
# LGR.fit(PCA3_train, y_train)
# predictions = LGR.predict(PCA3_test)
# pickle.dump(LGR, open("/content/drive/My Drive/Crypto/CryptoLGR3_model.sav", "wb"))
# np.savetxt("/content/drive/My Drive/Crypto/LGR3_X.txt",predictions)

# # 3 component XGBC
# XGBC = XGBClassifier(max_depth = 2, learning_rate=.5, random_state=69)
# XGBC.fit(PCA3_train, y_train)
# predictions = XGBC.predict(PCA3_test)
# np.savetxt("/content/drive/My Drive/Crypto/XGB3_predictions.txt", predictions)
# pickle.dump(XGBC, open("/content/drive/My Drive/Crypto/XGBC3_model.sav", "wb"))


# #-----------------------------------------------------


# #4 components
# LGR = LogisticRegression(class_weight = 'balanced', random_state = 69)
# pca = PCA(n_components=4)
# PCA4_train = pca.fit_transform(X_train)
# PCA4_test = pca.fit_transform(X_test)
# LGR.fit(PCA4_train, y_train)
# predictions = LGR.predict(PCA4_test)
# np.savetxt("/content/drive/My Drive/Crypto/LGR4_X.txt",predictions)
# pickle.dump(LGR, open("/content/drive/My Drive/Crypto/CryptoLGR4_model.sav", "wb"))
# #4 components XGBC
# XGBC = XGBClassifier(max_depth = 2, learning_rate=.5, random_state=69)
# XGBC.fit(PCA4_train, y_train)
# predictions = XGBC.predict(PCA4_test)
# np.savetxt("/content/drive/My Drive/Crypto/XGB4_predictions.txt", predictions)
# pickle.dump(XGBC, open("/content/drive/My Drive/Crypto/XGBC4_model.sav", "wb"))

In [ ]:
# This takes over 2 hours. I'm not going to do it now. Maybe if I upgrade my computer, I'll do it
#This is why I'm only doing 1
import time
time_start = time.time()
UX_train = umap.UMAP(n_neighbors=3, min_dist=0.5, metric='correlation').fit_transform(X_train)
UX_test = umap.UMAP(n_neighbors=3, min_dist=0.5, metric='correlation').fit_transform(X_test)
#umap is taking forever so I'm just going to do one iteration of it. 3 
print('UMAP done! Time elapsed: {} seconds'.format(time.time()-time_start))

# Final conclusions


Hypothesis: Rejection! I don't make money. 

# WHY?

----------------------------------------------------------------------

# THIS IS MY METRIC! HOW MUCH MONEY WAS MADE IN A WEEK

while precision (% correctly predicted buy) recall(%buy predicted over total buy) accuracy, and F1 (mean of precision and recall) are important, investors don'care more about how much money the model makes. Therefore, I am focusing in on amount made as the main metric. The other metrics can be used to better understand and tune models, but for investors, they just want to know how much money the models will make them.

---------------------------------------------------------------------

If I invested $1000, how much would I end up with in a week?

If prediction was 100% correct: $1070.95

Logistic Regression: $982.93

So I would lose money if I were to follow any of my models. Therefore, I reject the hypothesis

--------------------------------------------------------------------------------


creativity in features = lack of earned money. The feature reduction methods don't even perfom better than the base signal.
- The simple up/down prediction loses around 1% per day
- The Bollinger band strategy also loses X% per day
- The youtube guru strategy loses money at x% per day

The top two strategies were things that I thought of. The final strategy is linked below

guru strategy = "https://www.youtube.com/watch?v=mqqx4cGyUoY "


There is a bunch of room for improvement (obviously) but that's for another day
